<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [37]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [38]:
import folium
import pandas as pd

In [39]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [40]:
import numpy as np # useful for many scientific computing in Python import pandas as pd # primary data structure library

In [41]:
#!pip3 install folium==0.5.0
import folium
print('Folium installed and imported!')

Folium installed and imported!


In [42]:
import pandas as pd

# Load the dataset from its folder
spacex_df = pd.read_csv("spacex_launch_geo.csv")

# Show the first few rows
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [43]:
## Task 1: Mark all launch sites on a map

# Extrair sites e coordenadas (pega apenas 1 linha por site)
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df = launch_sites_df.groupby('Launch Site', as_index=False).first()

# Criar o mapa centralizado nos EUA
site_map = folium.Map(location=[30, -95], zoom_start=4)

# Adicionar marcadores para cada launch site
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Mostrar mapa
site_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [44]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [45]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [46]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [47]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [48]:
# Initialize the map centered at NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for each launch site
for index, row in launch_sites_df.iterrows():
    # Coordinates of the site
    coordinate = [row['Lat'], row['Long']]
    
    # Add circle
    circle = folium.Circle(
        location=coordinate,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add marker with label
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [49]:
# Task 2: Mark the success/failed launches for each site on the map

# Initialize the map centered at NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
launch_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add markers for each launch record
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Color based on success (1) or failure (0)
    color = 'green' if row['class'] == 1 else 'red'
    
    folium.CircleMarker(
        location=coordinate,
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{row['Launch Site']} | Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(launch_map)

launch_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [50]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [51]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [52]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column marker_color based on class
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Check the last few rows to confirm
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [53]:
# Initialize the MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each row in spacex_df
for index, record in spacex_df.iterrows():
    # Create the marker
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=record['Launch Site'],   # popup shows launch site name
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket', prefix='fa')
    )
    # Add the marker to the cluster
    marker_cluster.add_child(marker)

# Display map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [59]:
from math import radians, sin, cos, sqrt, atan2

# Função para calcular distância geodésica em km
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # raio da Terra em km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [60]:
# Launch sites
launch_sites = {
    "CCAFS LC-40": (28.562302, -80.577356),
    "CCAFS SLC-40": (28.563197, -80.576820),
    "KSC LC-39A": (28.573255, -80.646895),
    "VAFB SLC-4E": (34.632834, -120.610745)
}

# Pontos de interesse (coastline, highway, city)
points_of_interest = {
    "Coastline": (28.563, -80.567),   # Perto de Cabo Canaveral
    "Highway": (28.572, -80.799),     # Estrada próxima a KSC
    "City (Orlando)": (28.5383, -81.3792),  # Cidade grande próxima
    "City (Lompoc)": (34.6391, -120.4579)   # Próxima de VAFB
}

In [62]:
# Criar mapa centralizado nos EUA
distance_map = folium.Map(location=[30, -95], zoom_start=4)

# Iterar sobre os launch sites
for site_name, (site_lat, site_lon) in launch_sites.items():
    # Adicionar marcador do site
    folium.Marker([site_lat, site_lon],
                  popup=site_name,
                  icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
                 ).add_to(distance_map)
    
    # Iterar sobre pontos de interesse
    for poi_name, (poi_lat, poi_lon) in points_of_interest.items():
        # Calcular distância
        dist_km = calculate_distance(site_lat, site_lon, poi_lat, poi_lon)
        
        # Adicionar marcador do POI
        folium.Marker([poi_lat, poi_lon],
                      popup=f"{poi_name}",
                      icon=folium.Icon(color="red", icon="info-sign")
                     ).add_to(distance_map)
        
        # Desenhar linha conectando o site ao POI
        folium.PolyLine(
            locations=[[site_lat, site_lon], [poi_lat, poi_lon]],
            color="green", weight=2.5, opacity=0.8,
            tooltip=f"{site_name} → {poi_name}: {dist_km:.1f} km"
        ).add_to(distance_map)

# Mostrar mapa
distance_map
# No final do código, em vez de só "distance_map"
distance_map.save("launch_sites_distance_map.html")

In [67]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Dicionário de pontos de interesse (exemplo)
poi = {
    "Orlando": (28.5383, -81.3792),
    "Miami": (25.7617, -80.1918),
    "Los Angeles": (34.0522, -118.2437)
}

# Loop para calcular distâncias do site para cada POI
for name, (lat, lon) in poi.items():
    dist = calculate_distance(site_lat, site_lon, lat, lon)
    print(f"Distância do KSC LC-39A até {name}: {dist:.2f} km")

Distância do KSC LC-39A até Orlando: 3753.42 km
Distância do KSC LC-39A até Miami: 3979.61 km
Distância do KSC LC-39A até Los Angeles: 226.70 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [68]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [69]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [70]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# ---- Preencha / rode isto após usar MousePosition para pegar as coordenadas costeiras ----

from folium.features import DivIcon
from IPython.display import IFrame  # opcional para visualização segura
from math import radians, sin, cos, sqrt, atan2

# (Re)defina a função de distância (haversine)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# --- Garanta que launch_sites_df exista; se não, crie a partir do spacex_df ---
try:
    launch_sites_df
except NameError:
    launch_sites_df = spacex_df[['Launch Site','Lat','Long']].drop_duplicates().reset_index(drop=True)

# --- Escolha o launch site (substitua pelo site que você quer analisar) ---
site_name = "CCAFS LC-40"   # <-- troque para 'KSC LC-39A', 'VAFB SLC-4E', etc. conforme quiser

# Pegar coordenadas do site escolhido
site_row = launch_sites_df[launch_sites_df['Launch Site'] == site_name]
if site_row.empty:
    raise ValueError(f"Launch site '{site_name}' não encontrado em launch_sites_df.")
launch_site_lat = float(site_row['Lat'].iloc[0])
launch_site_lon = float(site_row['Long'].iloc[0])

# --- COORDENADAS DO PONTO COSTEIRO (pegue com MousePosition e cole aqui) ---
# Exemplo (substitua pelos valores que você obteve): Lat: 28.56367  Lon: -80.57163
coastline_lat = 28.56367   # <-- cole aqui a latitude mostrada pelo MousePosition
coastline_lon = -80.57163  # <-- cole aqui a longitude mostrada pelo MousePosition

# Calcular distância
distance_coastline_km = calculate_distance(launch_site_lat, launch_site_lon,
                                           coastline_lat, coastline_lon)

print(f"Distância do site '{site_name}' até a costa: {distance_coastline_km:.3f} km")

# --- Adicionar ao mapa já existente (site_map) ---
# Se você estiver usando outro mapa (por exemplo site_map ou distance_map), substitua o nome.
# Aqui eu uso 'site_map' porque você já vinha usando esse objeto.
try:
    site_map
except NameError:
    site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Marcador do ponto costeiro com DivIcon mostrando a distância
coast_coordinate = [coastline_lat, coastline_lon]
distance_marker = folium.Marker(
    location=coast_coordinate,
    icon=DivIcon(
        icon_size=(180,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_coastline_km:0.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

# Linha ligando o launch site ao ponto costeiro
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="blue", weight=2.5, opacity=0.8,
    tooltip=f"{site_name} → coastline: {distance_coastline_km:.2f} km"
).add_to(site_map)

# (Opcional) adicionar/garantir que o marcador do launch site exista
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup=site_name,
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(site_map)

# Salvar em HTML (útil se o notebook não estiver Trusted)
html_path = "task3_coastline_distance.html"
site_map.save(html_path)
print(f"Mapa salvo em: {html_path}  — você pode abrir no navegador ou usar IFrame abaixo para visualizar.")

# (Opcional) embutir via IFrame para visualizar inline sem depender de Trust Notebook:
# IFrame(html_path, width=900, height=600)

# Mostrar o mapa no notebook (se seu Jupyter permitir)
site_map

Distância do site 'CCAFS LC-40' até a costa: 0.580 km
Mapa salvo em: task3_coastline_distance.html  — você pode abrir no navegador ou usar IFrame abaixo para visualizar.


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [73]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="blue", weight=2.5, opacity=0.8,
    tooltip=f"{site_name} → coastline: {distance_coastline_km:.2f} km"
).add_to(site_map)

In [74]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
